In [2]:
import os
import ast
import pandas as pd

In [3]:
#INITIAL FILTER - Sirisha's file:racist_processed.csv

def tweet_dicts_csv(filename):
    max_tweets = 50000
    ind = 0
    tweet_dicts = []
    df = pd.read_csv(filename)
    df.shape

    for index_val, row in df.iterrows(): 
        tweet_dict = {}
        tweet_id = row["id"]
        tweet_text = row["text"]
        tweet_dict["tweet_id"] = tweet_id
        tweet_dict["tweet_text"] = tweet_text
        tweet_dicts.append(tweet_dict)
    print(len(tweet_dicts))
    
    return tweet_dicts
csv_filename = "racist_processed.csv"
tweet_dicts_csv = tweet_dicts_csv(csv_filename)

3060


In [2]:
#INTITIAL FILTER - Tweets provided by TA staff

def tweet_dicts_txt(filename):
    """ returns list of tweets that were not retweeted or incomplete"""
    max_tweets = 50000
    ind = 0
    tweet_dicts = []
    with open(filename) as f:
        for line in f:
            #print(line)
            dict_line = ast.literal_eval(line) #evaluates strings into dicts

            tweet_text = dict_line["tweet_text"]
            if not tweet_text.endswith("...") and not tweet_text.startswith("RT"):
            #if not tweet_text.endswith("..."):
                tweet_dicts.append(dict_line)
                #print(dict_line)
                ind += 1
                if ind ==max_tweets:
                    break
    print("NUMBER OF TWEETS - COMPLETE AND NON RT: ", len(tweet_dicts))
    return tweet_dicts

txt_filename = "US_data_about_covid.txt"
tweet_dicts_txt = tweet_dicts_txt(txt_filename)

NUMBER OF TWEETS - COMPLETE AND NON RT:  25552


In [37]:
#filter tweets for tweets that contain any wanted hashtags 
wanted_keywords = set(["chinavirus", "chinesevirus", "chinaliedandpeopledied", 
                       "covidiots", "china","chinese",
                      "chink", "asian", "chinaliedpeopledied", "wuhanvirus", "ccpvirus",
                      "chinesebioterrorism", "boycottchina", "coronavirustruth", "ccp_is_terrorist",
                      "wuflu", "chinesewuhanvirus", 
                      "keepamericagreat", "maga", "makeamericagreatagain", "trump2020", "holdchinaaccountable", "makechinapay", 
                      "racist", "racism",  "ccp",
                      "wholiedpeopledied",  "crime", "propoganda",
                      "communist", "communism"])

def filter_keyword_and_mentions(tweet_dicts, wanted_keywords):
    ind = 0
    china_tweets = []
    ht_or_mn_threshold = 0
    for i in range(len(tweet_dicts)):
        tweet = tweet_dicts[i]
        text = tweet["tweet_text"]
        text_tokens = text.split()
        tweet_hashtags = set()
        wanted_tweets = 0
        hashtag_or_mention = 0
        total_tokens = len(text_tokens)
        for token in text_tokens:
            if token.startswith("#"):
                hashtag_or_mention += 1
                hashtag_text = token[1:].lower()
                tweet_hashtags.add(hashtag_text)

            if token.startswith("@"):
                hashtag_or_mention += 1

            if token.lower() in wanted_keywords:
                tweet_hashtags.add(token.lower())
        wanted_tweets = len(tweet_hashtags & wanted_keywords)
        percent_ht_or_mn = hashtag_or_mention/total_tokens

        if wanted_tweets > 0:
            if percent_ht_or_mn <=0.75:
                china_tweets.append(tweet)

    print("NUMBER OF TWEETS - CONTAINING KEYWORDS: ", len(china_tweets))
    
    return china_tweets
    
china_tweets = filter_keyword_and_mentions(tweet_dicts, wanted_keywords)

NUMBER OF TWEETS - CONTAINING KEYWORDS:  2988


In [38]:
#remove the ones that mostly are links

def filter_url(china_tweets):
    ind = 0
    link_threshold = 0.5

    final_tweets = []

    for i in range(len(china_tweets)):
        tweet = china_tweets[i]
        text = tweet["tweet_text"]
        text_tokens = text.split()

        text_tokens = [x.lower() for x in text_tokens]      
        link_token = ""

        for token in text_tokens:
            if token.startswith("https://"):
                link_token += token
                link_percentage = len(link_token)/len(text)
                if link_percentage <= 0.5:
                    final_tweets.append(tweet)

    print("LENGTH OF TWEETS - GOOD PERCENTAGE: ", len(final_tweets))
    
    return final_tweets

final_tweets = filter_url(china_tweets)

LENGTH OF TWEETS - GOOD PERCENTAGE:  1329


In [39]:
#Save to csv file - TA tweets
ind = 0
added_text = set()
csv_tweets = []
for i in range(len(final_tweets)):
    #print(i)
    tweet = final_tweets[i]
    #print(tweet)
    tweet_text = tweet["tweet_text"]
    text_content = tweet_text.split("https://")[0]
    if text_content not in added_text:
        #print("----")
        added_text.add(text_content)
        csv_tweets.append(tweet)
    #print(text_tokens)
    
    #text_tokens = [x.lower() for x in text_tokens]  
print("LENGTH OF TWEETS - NON DUPLICATED CONTENT: ", len(csv_tweets))
    

LENGTH OF TWEETS - NON DUPLICATED CONTENT:  1213


In [47]:
#save to csv - Sirisha's tweets
ind = 0
import csv
#output_filename = "covid_tweets.csv"
max_count = 1213
output_filename = "racist_preprocessed_filtered_" + str(max_count) + ".csv"
header = 'tweet_id, tweet_text'

with open(output_filename, "w") as file:
    writer = csv.writer(file)
    
    writer.writerow(["tweet_id","tweet_text"])
    for i in range(len(csv_tweets)):
        tweet = csv_tweets[i]
        tweet_id = tweet["tweet_id"]
        tweet_text = tweet["tweet_text"]
        try:
            writer.writerow([tweet_id, tweet_text])
            ind += 1
        except:
            pass
        if ind == max_count:
            break
        
print("ADDED TWEETS: ", ind)

ADDED TWEETS:  1213
